## A look at *jeff* from Python

In this demonstration, we'll explore what it would look like to integrate *jeff* into a Python-based
workflow. Instead of using an existing quantum compilation framework like TKET or Catalyst, this
use-case assumes that we have some custom data structures at our disposal for representing and
manipulating quantum programs, and that we'd like to apply one of our optimizations on a program
exchanged with *jeff*.

Generally, cap'n proto can generate bindings in various languages to read and write *jeff* messages,
however, for Python these bindings are fully dynamic which makes it more difficult to discover the
API. Instead, we'll use the custom Python bindings from the *jeff* repository, which makes some
ergonomic improvements, while still mimicking most of the structure in the schema.

For comparison, let's take this simple example from the repository and load it through the generated bindings:

In [1]:
# avoid warning if this env variable is defined and different than the jupyter cwd
import os
if "PWD" in os.environ:
    del os.environ["PWD"]

# generic bindings from the pycapnp package
import capnp

capnp.remove_import_hook()
schema = capnp.load("impl/capnp/jeff.capnp")

with open("examples/catalyst_simple/catalyst_simple.jeff", "rb") as f:
    module = schema.Module.read(f)

print(module)

( version = 0,
  functions = [
    ( name = 0,
      definition = (
        body = (
          sources = [],
          targets = [9],
          operations = [
            ( inputs = [],
              outputs = [0],
              instruction = (int = (const32 = 0)) ),
            ( inputs = [],
              outputs = [1],
              instruction = (int = (const32 = 5)) ),
            ( inputs = [],
              outputs = [2],
              instruction = (float = (const64 = 0.1)) ),
            ( inputs = [1],
              outputs = [3],
              instruction = (qureg = (alloc = void)) ),
            ( inputs = [3, 0],
              outputs = [4, 5],
              instruction = (
                qureg = (extractIndex = void) ) ),
            ( inputs = [5],
              outputs = [6],
              instruction = (
                qubit = (
                  gate = (
                    custom = (name = 1, numQubits = 1, numParams = 0),
                    controlQubits = 0,
   

What we see is a textual representation of the loaded program, which matches the struct definitions
of the schema exactly, and is somewhat reminiscend of json or similar text data formats. While
this representation *can* be turned back into a binary encoding of the program, that would defeat
one of the main advatanges of *jeff*.

As far as readbility goes, it's a bit verbose and requires some crossreferencing. Let's use the
dedicated *jeff* bindings, which includes some pretty printing:

In [2]:
# assumes cwd is the repo root, adjust as needed or pip install
import sys
sys.path.insert(0, os.getcwd() + "/impl/py")

# the custom jeff bindings maintain the zero-copy nature of the capnp bindings when used for reading
import jeff

module = jeff.load_module("examples/catalyst_simple/catalyst_simple.jeff")
print(module)

jeff v0

[entry] func @hello() -> (int1):
  in :
    %0:int64 = int.const32  0
    %1:int64 = int.const32  5
    %2:float64 = float.const64  0.1
    %3:qureg = qureg.alloc %1:int64
    %4:qureg, %5:qubit = qureg.extractIndex %3:qureg, %0:int64
    %6:qubit = qubit.gate %5:qubit ("Hadamard", numQubits=1)
    %7:qubit = qubit.gate %6:qubit, %2:float64 ("RY", numQubits=1, numParams=1)
    %8:qubit, %9:int1 = qubit.measureNd %7:qubit
    %10:qureg = qureg.insertIndex %4:qureg, %8:qubit, %0:int64
    qureg.free %10:qureg
  out: %9:int1
func @world() -> ():
  in :
  out:



Much better! We can easily read off the various components of the loaded program, like the functions
and operations defined within, as well as metadata like the jeff version, or which function is the
entry point.

Another core design feature of *jeff* is apparent here as well, which is that instructions are
already arranged in a graph structure through the inputs and outputs of operations (marked with
`%`), which represent the edges in a dataflow graph. Note that qubit values in *jeff* are actually
a linear type, so there is always *exactly* one use for every qubit value.

It's important to note that these textual representations are merely for human inspectibility,
when connecting compilation tools together via *jeff* a main advantage is that this can be done
entirely with a binary representation. The cap'n proto binary encoding is such that any part of it
can be read directly from memory [without copying or decoding](https://capnproto.org/) the entire
message first.

### Optimizing a structured program

Let's get back to the main objective of the demo. Let's say we want to apply an optimization we
came up with to a program from another tool. To make things interesting, we'll consider an
optimization of quantum gates across control flow boundaries - after all, one of the core design
elements in *jeff* is the representation of structured, hybrid quantum programs.

Our (simple) optimization will match the following pattern and cancel the pair of Hadamards across
the conditional, from:

```py
H(i)
if b:
  H(i)
  ...
```

into:

```py
if b:
  ...
else:
  H(i)
```

As the first step, we would load a given program as before, in order to convert it to our own
data structures:

In [3]:
import jeff

module = jeff.load_module("examples/python_optimization/python_optimization.jeff")
print(module)

jeff v0

[entry] func @main(int1) -> ():
  in : %0:int1
    %1:qubit = qubit.alloc 
    %2:qubit = qubit.gate %1:qubit (h)
    %3:int1 = int.not %0:int1
    %4:qubit = scf.switch %3:int1, %2:qubit 
      case 0:
        in : %5:qubit
          %6:qubit = qubit.gate %5:qubit (h)
        out: %6:qubit
      default:
        in : %7:qubit
        out: %7:qubit
    qubit.free %4:qubit
  out:



However, for the purposes of this demo we will just use the *jeff* builder bindings directly, which
will also allow us to see how to construct a *jeff* program. It should be noted hower that this is
not an intended use case of *jeff*, in fact, manipulating programs is an explicit non-goal.

Let's reconstruct the program we see above using the custom Python bindings. Generally, types,
values, regions, functions, and modules are constructed directly by instantiating the corresponding
classes, however, to instantiate operations we use additional helper functions for convenience.

<!-- # for the optimization example we want a different circuit than the above
# one way to get it is to generate it from python
# this builder api is mostly composed of dataclasses that mirror the capnp schema,
# however some things are abstracted away or simplified
# additionally, high-level functions are added to aid in the construction of program objects
# Note: atm no encoding functionality to canpn is present -->

In [4]:
from jeff import *

func_inputs = [JeffValue(IntType(1))]

alloc = qubit_alloc()
h = quantum_gate("h", qubits=alloc.outputs[0])

bitnot = bitwise_not(func_inputs[0])

# true block
switch_arg = JeffValue(QubitType())
inner_h = quantum_gate("h", qubits=switch_arg)
then_region = JeffRegion(
    sources=[inner_h.inputs[0]], targets=[inner_h.outputs[0]], operations=[inner_h]
)
# false block
switch_arg = JeffValue(QubitType())
else_region = JeffRegion(sources=[switch_arg], targets=[switch_arg], operations=[])

switch = switch_case(
    index=bitnot.outputs[0], region_args=[h.outputs[0]], branches=[then_region], default=else_region
)
free = qubit_free(switch.outputs[0])

func_body = JeffRegion(sources=func_inputs, targets=[], operations=[alloc, h, bitnot, switch, free])
func = FunctionDef(name="main", body=func_body)

mod = JeffModule([func])
print(mod)
# TODO: could print some sentinel (-1, None, ...) instead of id for non_encoded values, but ...

jeff v0

[entry] func @main(int1) -> ():
  in : %4397829072:int1
    %4402776464:qubit = qubit.alloc 
    %4402736208:qubit = qubit.gate %4402776464:qubit (h)
    %4402834256:int1 = int.not %4397829072:int1
    %4404055888:qubit = scf.switch %4402834256:int1, %4402736208:qubit 
      case 0:
        in : %4404309008:qubit
          %4404304208:qubit = qubit.gate %4404309008:qubit (h)
        out: %4404304208:qubit
      default:
        in : %4404302800:qubit
        out: %4404302800:qubit
    qubit.free %4404055888:qubit
  out:



As we can see we re-created the same program as the one we loaded originally, the only difference
is that program values have some random-looking values assigned to them. This is the Python id of
the value object, which is a local property. To obtain the nicer print-out from before with globally
assigned labels for the value, we can encode the program into cap'n proto and generate a value
table in the process:

In [5]:
mod.refresh()  # re-encode a new cap'n proto message
print(mod)

jeff v0

[entry] func @main(int1) -> ():
  in : %0:int1
    %1:qubit = qubit.alloc 
    %2:qubit = qubit.gate %1:qubit (h)
    %3:int1 = int.not %0:int1
    %4:qubit = scf.switch %3:int1, %2:qubit 
      case 0:
        in : %5:qubit
          %6:qubit = qubit.gate %5:qubit (h)
        out: %6:qubit
      default:
        in : %7:qubit
        out: %7:qubit
    qubit.free %4:qubit
  out:



If we wish to write the encoded program to disk, we can use the `write_out` function like so:
```py
mod.write_out("examples/python_optimization/python_optimization.jeff")
```

Let's now implement the optimization we described earlier. Because the pattern is relatively simple
we can write out bespoke matching logic for it, but the code below is also a good demonstration for
why compiler frameworks with dedicated pattern matching systems are really useful.

In [6]:
# Start with a copy to avoid mutating the original program.
from copy import deepcopy

transformed_mod = deepcopy(mod)
for func in transformed_mod:
    match = None
    for op in func:
        # look for a first hadamard (h) gate to match against
        if not match:
            if not isinstance(op.instruction_data, WellKnowGate):
                continue

            gate = op.instruction_data
            if gate.kind != "h":
                continue

            match = op
            continue

        # let's check if h is followed by a conditional
        h_wire = match.outputs[0]
        if h_wire not in op.inputs:
            continue

        # the conditional is represented as a switch with only two branches
        if not isinstance(op.instruction_data, SwitchSCF):
            continue

        switch = op.instruction_data
        if len(switch.branches) != 1 or not switch.default:
            continue

        then_branch = switch.branches[0]
        else_branch = switch.default

        # check if there is a hadamard in the conditional
        wire_idx_in_switch_args = op.inputs.index(h_wire) - 1  # -1 for switch index
        then_wire = then_branch.sources[wire_idx_in_switch_args]
        nested_match = None
        wire_to_replace = None
        for nested_op in then_branch:
            # match a second hadamard op
            if not nested_match:
                if not isinstance(nested_op.instruction_data, WellKnowGate):
                    continue
                if nested_op.instruction_data.kind != "h":
                    continue
                if nested_op.inputs[0] != then_wire:
                    continue

                nested_match = nested_op
                wire_to_replace = nested_match.outputs[0]
                continue

            # we found a match!
            # let's remove the op we found by replacing its output wire with its input wire
            # first we'll handle the case where the wire is being used by another op
            if wire_to_replace not in nested_op.inputs:
                continue

            nested_wire_idx = nested_op.inputs.index(wire_to_replace)
            nested_op.inputs[nested_wire_idx] = then_wire
            wire_to_replace = None  # mark task as done

        # otherwise, the output wire was not used further in the branch and would have been returned
        if wire_to_replace:
            assert wire_to_replace in then_branch.targets
            out_idx = then_branch.targets.index(wire_to_replace)
            then_branch.targets[out_idx] = then_wire
            wire_to_replace = None  # mark task as done

        # delete the nested h gate from the then branch
        operations = then_branch.operations
        operations.remove(nested_match)
        then_branch.operations = operations
        nested_match = None  # mark task as done

        # insert a new h gate into the else branch
        else_wire = else_branch.sources[wire_idx_in_switch_args]
        h_gate = quantum_gate("h", else_wire)
        operations = else_branch.operations
        operations.insert(0, h_gate)
        else_branch.operations = operations

        # update the existing use of the wire
        # first we'll handle the case where the wire is being used by another op
        replacement_wire = h_gate.outputs[0]
        for nested_op in else_branch.operations[1:]:  # skip the h gate we just inserted
            if else_wire not in nested_op.inputs:
                continue

            nested_wire_idx = nested_op.inputs.index(else_wire)
            nested_op.inputs[nested_wire_idx] = replacement_wire
            replacement_wire = None  # mark task as done

        # otherwise, the output wire was not used further in the branch and would have been returned
        if replacement_wire:
            assert else_wire in else_branch.targets
            out_idx = else_branch.targets.index(else_wire)
            else_branch.targets[out_idx] = replacement_wire
            replacement_wire = None  # mark task as done

        # let's get ready to delete the original h gate by rewiring its input to the output
        h_wire_idx = op.inputs.index(h_wire)
        op.inputs[h_wire_idx] = match.inputs[0]

        break

    # delete the original h gate
    operations = func.body.operations
    operations.remove(match)
    func.body.operations = operations
    match = None  # mark task as done

# refresh internal data structures
transformed_mod.refresh()
print(transformed_mod)

jeff v0

[entry] func @main(int1) -> ():
  in : %0:int1
    %1:qubit = qubit.alloc 
    %2:int1 = int.not %0:int1
    %3:qubit = scf.switch %2:int1, %1:qubit 
      case 0:
        in : %4:qubit
        out: %4:qubit
      default:
        in : %5:qubit
          %6:qubit = qubit.gate %5:qubit (h)
        out: %6:qubit
    qubit.free %3:qubit
  out:



Great, we have successfully applied the optimization. Let's look at it side-by-side:

In [7]:
def print_side_by_side(left_obj, right_obj):
    """Print two objects side by side using their string representations."""
    left_str = str(left_obj)
    right_str = str(right_obj)

    left_lines = left_str.splitlines()
    right_lines = right_str.splitlines()

    width = max(max(len(l) for l in left_lines), max(len(r) for r in right_lines))
    
    max_lines = max(len(left_lines), len(right_lines))
    left_lines += [''] * (max_lines - len(left_lines))
    right_lines += [''] * (max_lines - len(right_lines))
    
    for l, r in zip(left_lines, right_lines):
        print(f"{l:<{width}} | {r}")

# Test it with your objects
print("BEFORE vs AFTER")
print("=" * 91)
print_side_by_side(mod, transformed_mod)

BEFORE vs AFTER
jeff v0                                      | jeff v0
                                             | 
[entry] func @main(int1) -> ():              | [entry] func @main(int1) -> ():
  in : %0:int1                               |   in : %0:int1
    %1:qubit = qubit.alloc                   |     %1:qubit = qubit.alloc 
    %2:qubit = qubit.gate %1:qubit (h)       |     %2:int1 = int.not %0:int1
    %3:int1 = int.not %0:int1                |     %3:qubit = scf.switch %2:int1, %1:qubit 
    %4:qubit = scf.switch %3:int1, %2:qubit  |       case 0:
      case 0:                                |         in : %4:qubit
        in : %5:qubit                        |         out: %4:qubit
          %6:qubit = qubit.gate %5:qubit (h) |       default:
        out: %6:qubit                        |         in : %5:qubit
      default:                               |           %6:qubit = qubit.gate %5:qubit (h)
        in : %7:qubit                        |         out: %6:qubit
     

From here, we can pass the encoded (optimized!) program back to another tool.

Check out https://github.com/unitaryfoundation/jeff for more.